In [ ]:
import numpy as np
import matplotlib.pyplot as plt

sample = np.random.normal(size=1000)

(counts, bins) = np.histogram(sample)

bsz = bins[1] - bins[0]
bins = (bins + bsz / 2)[:-1]

bars = np.full_like(counts, 10)

plt.errorbar(bins, counts, yerr = bars, marker='.', linestyle='none')
plt.show()